# Data

In [2]:
planning_horizon = 480*60
directory = 'SMALL'
maxNumSet = 1
ReworkProbability = 0.03

In [188]:
import pandas as pd
import numpy as np

# Load the Excel file
PlanMaster                 = pd.read_excel("DataMaster.xlsx", sheet_name="Plan")
ComponentMaster            = pd.read_excel("DataMaster.xlsx", sheet_name="ComponentMaster")

Set                        = 48
Plan                       = PlanMaster[PlanMaster['Set'] == Set]
JobToSchedule              = ComponentMaster[ComponentMaster['ComponentID'].isin(Plan['ComponentID'])]
JobToSchedule['MachineID'] = JobToSchedule['MachineID'].astype(str)
OperationCount             = JobToSchedule['ComponentID'].value_counts().to_dict()
JobList                    = [component for _, row in Plan.iterrows() for component in [row['ComponentID']] * int(row['BatchedQuantity'])]

# Define total number of jobs (J) and total number of machines (K)
J = len(JobList)
K = 30

# Ensure all selected components are present in the OperationCount
for comp in JobList:
    if comp not in OperationCount:
        OperationCount[comp] = 0

# Defining a deadline for each component (planning horizon for simplicity)
Deadline = {i: planning_horizon for i in range(J)}

# Creating the text file in the specified format
output_file = f'{directory}/Case{Set}_{planning_horizon // 60}.txt'

with open(output_file, 'w') as f:
    f.write(f"{J}\n")
    f.write(f"{K}\n")
    f.write(" ".join(str(OperationCount[comp_id]) for comp_id in JobList) + "\n")
    f.write(" ".join(str(Deadline[comp_id]) for comp_id in range(J)) + "\n")
    job_id = 1
    for comp_id in JobList:
        comp_df = JobToSchedule[JobToSchedule['ComponentID'] == comp_id]
        for op_id, row in enumerate(comp_df.itertuples(), start=1):
            machines = row.MachineID.replace(' ', '').split(',')
            for machine in machines:
                f.write(f"{job_id} {op_id} {machine} {row.BatchedDurationInSecond}\n")
        job_id += 1


print("Case generation complete.")


C:\Users\ASUS\AppData\Local\Temp\ipykernel_1328\4267451025.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  JobToSchedule['MachineID'] = JobToSchedule['MachineID'].astype(str)


Case generation complete.


# Scenario

In [3]:
def get_info(file_path):
    machine_set = set()
    with open(file_path, 'r') as file:
        data = file.readlines()
        J = int(data[0].strip())
        for line in data[4:]:
            parts   = line.split()
            machine = int(parts[2])
            machine_set.add(machine)
    machine_list = list(machine_set)
    
    return J, machine_list

import ast
import math
import random
import pandas as pd
import numpy as np
from scipy.stats import weibull_min


# Read the Excel file
WeibullDistribution = pd.read_excel('DataMaster.xlsx', sheet_name='Distribution')
descriptions = {
                    'A': 'urgent',
                    'B': 'normal',
                    'C': 'loose'
                }

ScenarioList      = ['_fixed_scenario', 'A', 'B', 'C', 'D', 'E', 'F', 'G']
TypeList          = ['A', 'B', 'C']

Set = '_fixed_instance'
sourcename        = f'{directory}/Case{Set}_{planning_horizon // 60}.txt'
J, machine_list   = get_info(sourcename)

for Scenario in ScenarioList:
    scenario_type = random.choice(TypeList)
    filename      = f'{directory}/Scenario{Scenario}_480.txt'
    # Create a text file to store the results
    with open(filename, 'w') as f:

        # Separate machine breakdown and defected jobs data
        f.write('Defected Jobs\n')
        f.write('JobID, Deadline_info, Description\n')
    
        # Determine description based on scenario
        description = descriptions[scenario_type]
        
        # Set deadline based on description
        if description == 'urgent':
            Deadline = np.random.randint(1, 11, size=J)
        elif description == 'normal':
            Deadline = np.full(J, planning_horizon)
        elif description == 'loose':
            Deadline = np.full(J, 2 * planning_horizon)
        
        # Generate defected jobs with 3% probability
        defected_jobs_indices = np.where(np.random.uniform(size=J) < ReworkProbability)[0] + 1  # One-indexed
        for job_id in defected_jobs_indices:
            deadline = Deadline[job_id - 1]  # Adjusting for one-indexing
            f.write(f'{job_id:<10}, {deadline:<10}, {description:<10}\n')

        f.write('\nMachineBreakdown\n')
        f.write('MachineID, BD_time, \tRe_time\n')
        # Iterate over each row in the DataFrame
        for _, row in WeibullDistribution.iterrows():
            machine_id = row['MachineID']
            if machine_id in machine_list:
                shape_up, loc_up, scale_up       = ast.literal_eval(row['ParameterUpTime'])
                shape_down, loc_down, scale_down = ast.literal_eval(row['ParameterDownTime'])
                
                # Calculate uptime and downtime
                t = -604800*2
                downtime = 0
                breakdown_time_list = []
                repairt_time_list = []
                while t < planning_horizon:
                    skip     = False
                    t       += downtime
                    uptime   = math.ceil(weibull_min.rvs(c=shape_up, loc=0,   scale=scale_up))*60 # change to second
                    count    = 0
                    while uptime < 300:
                        uptime   = math.ceil(weibull_min.rvs(c=shape_up, loc=0,   scale=scale_up))*60 # change to second
                        count += 1
                        if count == 10:
                            skip = True
                            break

                    downtime = math.ceil(weibull_min.rvs(c=shape_down, loc=0, scale=scale_down))*60 # change to second
                    t += uptime
                    if skip == True:
                        break
                    else:
                        if t > 0 and t < planning_horizon:
                            breakdown_time_list.append(t)
                            repairt_time_list  .append(downtime)
            
            # Write the machine breakdown results to the text file
            for i in range(len(breakdown_time_list)):
                f.write(f'{machine_id:<10}{breakdown_time_list[i]:<15.1f}{repairt_time_list[i]:<15.1f}\n')

    print("Scenario generation complete.")


Scenario generation complete.
Scenario generation complete.
Scenario generation complete.
Scenario generation complete.
Scenario generation complete.
Scenario generation complete.
Scenario generation complete.
Scenario generation complete.


In [4]:
def get_info(file_path):
    machine_set = set()
    with open(file_path, 'r') as file:
        data = file.readlines()
        J = int(data[0].strip())
        for line in data[4:]:
            parts   = line.split()
            machine = int(parts[2])
            machine_set.add(machine)
    machine_list = list(machine_set)
    
    return J, machine_list

import ast
import math
import random
import pandas as pd
import numpy as np
from scipy.stats import weibull_min


# Read the Excel file
WeibullDistribution = pd.read_excel('DataMaster.xlsx', sheet_name='Distribution')
descriptions = {
                    'A': 'urgent',
                    'B': 'normal',
                    'C': 'loose'
                }

ScenarioList      = ['small']
TypeList          = ['A', 'B', 'C']

Set = '_fixed_instance'
sourcename        = f'{directory}/Case{Set}_{planning_horizon // 60}.txt'
J, machine_list   = get_info("jobs_small.txt")

for Scenario in ScenarioList:
    scenario_type = random.choice(TypeList)
    filename      = f'{directory}/Scenario{Scenario}_480.txt'
    # Create a text file to store the results
    with open(filename, 'w') as f:

        # Separate machine breakdown and defected jobs data
        f.write('Defected Jobs\n')
        f.write('JobID, Deadline_info, Description\n')
    
        # Determine description based on scenario
        description = descriptions[scenario_type]
        
        # Set deadline based on description
        if description == 'urgent':
            Deadline = np.random.randint(1, 11, size=J)
        elif description == 'normal':
            Deadline = np.full(J, planning_horizon)
        elif description == 'loose':
            Deadline = np.full(J, 2 * planning_horizon)
        
        # Generate defected jobs with 3% probability
        defected_jobs_indices = np.where(np.random.uniform(size=J) < ReworkProbability)[0] + 1  # One-indexed
        for job_id in defected_jobs_indices:
            deadline = Deadline[job_id - 1]  # Adjusting for one-indexing
            f.write(f'{job_id:<10}, {deadline:<10}, {description:<10}\n')

        f.write('\nMachineBreakdown\n')
        f.write('MachineID, BD_time, \tRe_time\n')
        # Iterate over each row in the DataFrame
        for _, row in WeibullDistribution.iterrows():
            machine_id = row['MachineID']
            if machine_id in machine_list:
                shape_up, loc_up, scale_up       = ast.literal_eval(row['ParameterUpTime'])
                shape_down, loc_down, scale_down = ast.literal_eval(row['ParameterDownTime'])
                
                # Calculate uptime and downtime
                t = -604800*2
                downtime = 0
                breakdown_time_list = []
                repairt_time_list = []
                while t < planning_horizon:
                    skip     = False
                    t       += downtime
                    uptime   = math.ceil(weibull_min.rvs(c=shape_up, loc=0,   scale=scale_up))*60 # change to second
                    count    = 0
                    while uptime < 300:
                        uptime   = math.ceil(weibull_min.rvs(c=shape_up, loc=0,   scale=scale_up))*60 # change to second
                        count += 1
                        if count == 10:
                            skip = True
                            break

                    downtime = math.ceil(weibull_min.rvs(c=shape_down, loc=0, scale=scale_down))*60 # change to second
                    t += uptime
                    if skip == True:
                        break
                    else:
                        if t > 0 and t < planning_horizon:
                            breakdown_time_list.append(t)
                            repairt_time_list  .append(downtime)
            
            # Write the machine breakdown results to the text file
            for i in range(len(breakdown_time_list)):
                f.write(f'{machine_id:<10}{breakdown_time_list[i]:<15.1f}{repairt_time_list[i]:<15.1f}\n')

    print("Scenario generation complete.")


Scenario generation complete.
